# Assignment 2: Training the Fully Recurrent Network

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.


## Exercise 1: Data generation

There are two classes, both occurring with probability 0.5. There is one input unit. Only the first sequence element conveys relevant information about the class. Sequence elements at positions $t > 1$ stem from a Gaussian with mean zero and variance 0.2. The first sequence element is 1.0 (-1.0) for class 1 (2). Target at sequence end is 1.0 (0.0) for class 1 (2)

Write a function `generate_data` that takes an integer `T` as argument which represents the sequence length. Seed the `numpy` random generator with the number `0xDEADBEEF`. Implement the [Python3 generator](https://docs.python.org/3/glossary.html#term-generator) pattern and produce data in the way described above. The input sequences should have the shape `(T, 1)` and the target values should have the shape `(1,)`.

In [25]:
%matplotlib inline
import numpy as np
from scipy.special import expit as sigmoid
import matplotlib.pyplot as plt

class FullyRecurrentNetwork(object):
    def __init__(self, D, I, K):
        self.W = np.random.uniform(-0.01, 0.01, (I, D))
        self.R = np.random.uniform(-0.01, 0.01, (I, I))
        self.V = np.random.uniform(-0.01, 0.01, (K, I))
    
    def forward(self, x, y):
        # helper function for numerically stable loss
        def f(z):
            return np.log1p(np.exp(-np.absolute(z))) + np.maximum(0, z)
        
        # infer dims
        T, D = x.shape
        K, I = self.V.shape

        # init result arrays
        self.x = x
        self.y = y
        self.a = np.zeros((T, I))

        # iterate forward in time 
        # trick: access model.a[-1] in first iteration
        for t in range(T):
            self.a[t] = np.tanh(self.W @ x[t] + self.R @ self.a[t-1])
            
        self.z = model.V @ self.a[t]
        return y * f(-self.z) + (1-y) * f(self.z)

T, D, I, K = 10, 3, 5, 1
model = FullyRecurrentNetwork(D, I, K)
model.forward(np.random.uniform(-1, 1, (T, D)), 1)

def generate_data(T):
    rng = np.random.RandomState(0xDEADBEEF)
    while True:
        x = rng.normal(0.0, 0.2, (T,1))
        if x[0] > 0:
            x[0] = 1.0
            y = np.array([1.0])
        else:
            x[0] = -1.0
            y = np.array([0.0])
        yield x, y

data = generate_data(2)

## Exercise 2: Gradients for the network parameters
Compute gradients of the total loss 
$$
L = \sum_{t=1}^T L(t), \quad \text{where} \quad L(t) = L(z(t), y(t))
$$
w.r.t. the weights of the fully recurrent network. To this end, find the derivative of the loss w.r.t. the logits and hidden pre-activations first, i.e., 
$$
\psi^\top(t) = \frac{\partial L}{\partial z(t)} \quad \text{and} \quad \delta^\top(t) = \frac{\partial L}{\partial s(t)}.
$$
With the help of these intermediate results you should be able to compute the gradients w.r.t. the weights, i.e., $\nabla_W L, \nabla_R L, \nabla_V L$. 

*Hint: Take a look at the computational graph from the previous assignment to see the functional dependencies.*

*Remark: Although we only have one label at the end of the sequence, we consider the more general case of evaluating a loss at every time step in this exercise (many-to-many mapping).*

########## YOUR SOLUTION HERE ##########


We have to interpret the objects $\psi^\top(t)$ and $\delta^\top(t)$ differently as given. We think it is meant
$$
    \psi^\top(t) = \frac{\partial L(z(t), y(t))}{\partial z(t)}
$$
and likewise
$$
    \delta^\top(t) = \frac{\partial L(z(t), y(t))}{\partial s(t)}.
$$


We inspect the network from last time, i.e.
$$
s(t) = W x(t) + R a(t-1) \\
a(t) = \tanh(s(t)) \\
z(t) = V a(t) \\
\hat y(t) = \sigma(z(t))
$$
for $t \in \mathbb{N}, x(t) \in \mathbb{R}^{D}, s(t) \in \mathbb{R}^{I}, a(t) \in \mathbb{R}^{I}, z(t) \in \mathbb{R}^K, \hat y(t) \in \mathbb{R}^K$ and $W, R, V$ are real matrices of appropriate sizes and $\hat a(0) = 0$. For the loss we assume the Cross-Entropy loss, i.e. $L(z(t), y(t))=-\sum_{t=1}^T y(t) \log \sigma(z(t))$.

We get 
\begin{align*}
    \psi^\top(t) &= \frac{\partial L(t)}{\partial z(t)}\\
    &= - y(t) \frac{\partial}{\partial z(t)} \log \sigma(z(t))\\
    &= - y(t) \frac{1}{\sigma(z(t))} \frac{\partial}{\partial z(t)} \sigma(z(t))\\
    &= - y(t) \frac{1}{\sigma(z(t))} \sigma(z(t)) (1-\sigma(z(t)))\\
    &= - y(t) (1-\sigma(z(t)))
\end{align*}
and
\begin{align*}
    \delta^\top(t) &= \frac{\partial L(t)}{\partial s(t)}\\
    &= - y(t) \frac{\partial}{\partial s(t)} \log \sigma(z(t))\\
    &= - y(t) \frac{\partial}{\partial s(t)} \log \sigma(V a(t))\\
    &= - y(t) \frac{1}{\sigma(z(t))} \frac{\partial}{\partial s(t)} \sigma(V a(t))\\
    &= - y(t) \frac{1}{\sigma(z(t))} \sigma'(z(t)) V \frac{\partial}{\partial s(t)} a(t)\\
    &= - y(t) (1-\sigma(z(t))) \frac{\partial}{\partial s(t)} \left( V \tanh(s(t)) \right)\\
    &= \psi^\top(t) V \operatorname{diag} (1-\tanh^2(s(t)))\\
    &= \psi^\top(t) V \operatorname{diag} (1-a(t)^2).
\end{align*}


Using this, we can calculate the gradients w.r.t. to the weights as follows
\begin{align*}
    \left(\frac{\partial L}{\partial W}\right)^\top
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial W}\\
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial s(t)} \frac{\partial s(t)}{\partial W}\\
    &= \sum_{t=1}^T \delta^\top(t) \frac{\partial s(t)}{\partial W}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) \frac{\partial s(t)}{\partial W}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) x(t)^\top.
\end{align*}

For the gradient w.r.t. $R$ we get
\begin{align*}
    \left(\frac{\partial L}{\partial R}\right)^\top
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial R}\\
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial s(t)} \frac{\partial s(t)}{\partial R}\\
    &= \sum_{t=1}^T \delta^\top(t) \frac{\partial s(t)}{\partial R}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) \frac{\partial s(t)}{\partial R}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) a(t-1)^\top.
\end{align*}

For the gradient w.r.t. $V$ we get
\begin{align*}
    \left(\frac{\partial L}{\partial V}\right)^\top
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial V}\\
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial z(t)} \frac{\partial z(t)}{\partial V}\\
    &= \sum_{t=1}^T \psi^\top(t) \frac{\partial z(t)}{\partial V}\\
    &= \sum_{t=1}^T \psi^\top(t) \frac{\partial}{\partial V} V a(t)\\
    &= - \sum_{t=1}^T y(t) (1-\sigma(z(t))) a(t)^\top.
\end{align*}


##### second version #######


It holds that
\begin{align*}
    \psi^\top(t) &= \frac{\partial L(z(t), y(t))}{\partial z(t)}\\
    &= y(t) \sigma(-z(t)) + (1-y(t)) \sigma(z(t))\\
    &= \sigma(z(t)) - y(t) + y \sigma(-z(t)) - y \sigma(-z(t))\\
    &= \sigma(z(t)) - y(t)
\end{align*}

and

\begin{align*}
    \delta^\top(t) &= \frac{\partial L(z(t), y(t))}{\partial s(t)}\\
    &= \frac{\partial L(z(t), y(t))}{\partial z(t)} \frac{\partial z(t)}{\partial a(t)} \frac{\partial a(t)}{\partial s(t)}\\
    &= \psi^\top(t) V \operatorname{diag}(1-a(t)^2).
\end{align*}

Therefore

\begin{align*}
    \left(\frac{\partial L}{\partial W}\right)^\top
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial W}\\
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial s(t)} \frac{\partial s(t)}{\partial W}\\
    &= \sum_{t=1}^T \delta^\top(t) \frac{\partial s(t)}{\partial W}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) \frac{\partial s(t)}{\partial W}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) x(t)^\top
\end{align*}

and

\begin{align*}
    \left(\frac{\partial L}{\partial R}\right)^\top
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial R}\\
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial s(t)} \frac{\partial s(t)}{\partial R}\\
    &= \sum_{t=1}^T \delta^\top(t) \frac{\partial s(t)}{\partial R}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) \frac{\partial s(t)}{\partial R}\\
    &= \sum_{t=1}^T \psi^\top(t) V \operatorname{diag}(1-a(t)^2) a(t-1)^\top
\end{align*}

and

\begin{align*}
    \left(\frac{\partial L}{\partial V}\right)^\top
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial V}\\
    &= \sum_{t=1}^T \frac{\partial L(t)}{\partial z(t)} \frac{\partial z(t)}{\partial V}\\
    &= \sum_{t=1}^T \psi^\top(t) \frac{\partial z(t)}{\partial V}\\
    &= \sum_{t=1}^T \psi^\top(t) \frac{\partial}{\partial V} V a(t)\\
    &= - \sum_{t=1}^T y(t) (1-\sigma(z(t))) a(t)^\top.
\end{align*}

## Exercise 3: The backward pass
Write a function `backward` that takes a model `self` as argument. The function should compute the gradients of the loss with respect to all model parameters and store them to `self.dW`, `self.dR`, `self.dV`, respectively. 

In [26]:
def backward(self):
    ########## YOUR SOLUTION HERE ##########
    psi = sigmoid(self.V @ self.a.T) - self.y
    delta = psi.T @ self.V * (1 - self.a**2)
    self.dV = psi @ self.a
    self.dR = delta[1:].T @ self.a[:-1]                 #  skip the last element in a and roll back one step
    self.dW = delta.T @ self.x
    return self.dW, self.dR, self.dV

def backward_1(self):
    # Calculate psi for the output layer as psi(t) = σ(z(t)) - y(t)
    psi = sigmoid(model.V @ model.a[-1]) - model.y  # Equivalent to σ(z(t)) - y(t)

    # Calculate gradient for V (output layer weights)
    dV = np.outer(psi, model.a[-1])

    # Calculate delta for the hidden layer using the formula δ(t) = ψᵀ(t) V diag(1 - a(t)^2)
    delta = (psi @ model.V) * (1 - model.a[-1] ** 2)  # Applying tanh derivative for hidden layer

    # Compute gradients for R (hidden-to-hidden weights) and W (input-to-hidden weights)
    dR = np.outer(delta, model.a[-2])
    dW = np.outer(delta, model.x[-1])
    
    return dW, dR, dV

FullyRecurrentNetwork.backward = backward_1
model.backward()

(array([[ 0.00116458,  0.00128018,  0.0021587 ],
        [-0.00092281, -0.00101441, -0.00171054],
        [ 0.00021288,  0.00023401,  0.0003946 ],
        [-0.00096082, -0.0010562 , -0.00178101],
        [-0.00058857, -0.00064699, -0.00109099]]),
 array([[-3.99816054e-06,  2.79479999e-05,  2.21677058e-05,
          2.15678270e-05,  3.79543643e-05],
        [ 3.16812099e-06, -2.21458453e-05, -1.75655712e-05,
         -1.70902306e-05, -3.00748349e-05],
        [-7.30853524e-07,  5.10882292e-06,  4.05219993e-06,
          3.94254363e-06,  6.93796076e-06],
        [ 3.29863739e-06, -2.30581830e-05, -1.82892163e-05,
         -1.77942931e-05, -3.13138215e-05],
        [ 2.02063955e-06, -1.41247039e-05, -1.12033878e-05,
         -1.09002137e-05, -1.91818435e-05]]),
 array([[-0.00195853, -0.00250685, -0.00164818,  0.00309986,  0.00137899]]))

## Exercise 4: Gradient checking
Write a function `grad_check` that takes a model `self`, a float `eps` and another float `thresh` as arguments and computes the numerical gradients of the model parameters according to the approximation
$$
f'(x) \approx \frac{f(x + \varepsilon) - f(x - \varepsilon)}{2 \varepsilon}.
$$
If any of the analytical gradients are farther than `thresh` away from the numerical gradients the function should throw an error. 

In [27]:
def grad_check(self, eps, thresh):
    dW_approx = np.zeros_like(self.W)
    dV_approx = np.zeros_like(self.V)
    dR_approx = np.zeros_like(self.R)

    for i in range(self.W.shape[0]):
        for j in range(self.W.shape[1]):
            self.W[i,j] += eps
            loss_plus = self.forward(self.x, self.y)
            self.W[i,j] -= 2*eps
            loss_minus = self.forward(self.x, self.y)
            dW_approx[i,j] = (loss_plus - loss_minus) / (2*eps)
            self.W[i,j] += eps

    for i in range(self.R.shape[0]):
        for j in range(self.R.shape[1]):
            self.R[i,j] += eps
            loss_plus = self.forward(self.x, self.y)
            self.R[i,j] -= 2*eps
            loss_minus = self.forward(self.x, self.y)
            dR_approx[i,j] = (loss_plus - loss_minus) / (2*eps)
            self.R[i,j] += eps

    for i in range(self.V.shape[0]):
        for j in range(self.V.shape[1]):
            self.V[i,j] += eps
            loss_plus = self.forward(self.x, self.y)
            self.V[i,j] -= 2*eps
            loss_minus = self.forward(self.x, self.y)
            dV_approx[i,j] = (loss_plus - loss_minus) / (2*eps)
            self.V[i,j] += eps
    
    self.forward(self.x, self.y)
    dW, dR, dV = self.backward()


    print(f"dW (analytical):\n{dW}\n")
    print(f"dW_approx (numerical):\n{dW_approx}\n")
    print(f"dR (analytical):\n{dR}\n")
    print(f"dR_approx (numerical):\n{dR_approx}\n")
    print(f"dV (analytical):\n{dV}\n")
    print(f"dV_approx (numerical):\n{dV_approx}\n")

    print(f"Error dW: {np.linalg.norm(dW - dW_approx)}")
    print(f"Error dR: {np.linalg.norm(dR - dR_approx)}")
    print(f"Error dV: {np.linalg.norm(dV - dV_approx)}")

    assert np.linalg.norm(dW - dW_approx) < thresh
    assert np.linalg.norm(dR - dR_approx) < thresh
    assert np.linalg.norm(dV - dV_approx) < thresh

    print("Gradients are correct")
    

FullyRecurrentNetwork.grad_check = grad_check
model.grad_check(1e-7, 1e-7)

dW (analytical):
[[ 0.00116458  0.00128018  0.0021587 ]
 [-0.00092281 -0.00101441 -0.00171054]
 [ 0.00021288  0.00023401  0.0003946 ]
 [-0.00096082 -0.0010562  -0.00178101]
 [-0.00058857 -0.00064699 -0.00109099]]

dW_approx (numerical):
[[ 0.00114188  0.00127163  0.00216955]
 [-0.00095655 -0.0010272  -0.00169415]
 [ 0.00019036  0.00022544  0.00040563]
 [-0.00091931 -0.00104039 -0.00180134]
 [-0.00056611 -0.00063842 -0.00110204]]

dR (analytical):
[[-3.99816054e-06  2.79479999e-05  2.21677058e-05  2.15678270e-05
   3.79543643e-05]
 [ 3.16812099e-06 -2.21458453e-05 -1.75655712e-05 -1.70902306e-05
  -3.00748349e-05]
 [-7.30853524e-07  5.10882292e-06  4.05219993e-06  3.94254363e-06
   6.93796076e-06]
 [ 3.29863739e-06 -2.30581830e-05 -1.82892163e-05 -1.77942931e-05
  -3.13138215e-05]
 [ 2.02063955e-06 -1.41247039e-05 -1.12033878e-05 -1.09002137e-05
  -1.91818435e-05]]

dR_approx (numerical):
[[-3.98514555e-06  2.77378120e-05  2.20307106e-05  2.14767093e-05
   3.77819998e-05]
 [ 3.18911564e

/var/folders/z1/xfwm3xr90019fcl3by7y113c0000gn/T/ipykernel_18096/3583158363.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dW_approx[i,j] = (loss_plus - loss_minus) / (2*eps)
/var/folders/z1/xfwm3xr90019fcl3by7y113c0000gn/T/ipykernel_18096/3583158363.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dR_approx[i,j] = (loss_plus - loss_minus) / (2*eps)
/var/folders/z1/xfwm3xr90019fcl3by7y113c0000gn/T/ipykernel_18096/3583158363.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Depr

AssertionError: 

## Exercise 5: Parameter update

Write a function `update` that takes a model `self` and a float argument `eta`, which represents the learning rate. The method should implement the gradient descent update rule $\theta \gets \theta - \eta \nabla_{\theta}L$ for all model parameters $\theta$.

In [ ]:
def update(self, eta):
    ########## YOUR SOLUTION HERE ##########

FullyRecurrentNetwork.update = update
model.update(0.001)

## Exercise 6: Network training

Train the fully recurrent network with 32 hidden units. Start with input sequences of length one and tune the learning rate and the number of update steps. Then increase the sequence length by one and tune the hyperparameters again. What is the maximal sequence length for which the fully recurrent network can achieve a performance that is better than random? Visualize your results. 

In [ ]:
########## YOUR SOLUTION HERE ##########

## Exercise 7: The Vanishing Gradient Problem

Analyze why the network is incapable of learning long-term dependencies. Show that $\|\frac{\partial a(T)}{\partial a(1)}\|_2 \leq \|R\|_2^{T-1}$ , where $\|\cdot\|_2$ is the spectral norm, and discuss how that affects the propagation of error signals through the time dimension of the network. 

*Hint: Use the fact that the spectral norm is submultiplicative for square matrices, i.e. $\|AB\|_2 \leq \|A\|_2\|B\|_2$ if $A$ and $B$ are both square.*

########## YOUR SOLUTION HERE ##########